In [ ]:
import numpy as np
import tensorflow as tf

import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

# from tensorflow.python.keras.applications import ResNet50
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers import Dense

In [ ]:
!unzip /content/drive/MyDrive/dataset/newx64.zip

Streaming output truncated to the last 5000 lines.
  inflating: newx64/NotFaces/new (1895).jpg  
  inflating: newx64/NotFaces/new (1896).jpg  
  inflating: newx64/NotFaces/new (1897).jpg  
  inflating: newx64/NotFaces/new (1898).jpg  
  inflating: newx64/NotFaces/new (1899).jpg  
  inflating: newx64/NotFaces/new (19).jpg  
  inflating: newx64/NotFaces/new (190).jpg  
  inflating: newx64/NotFaces/new (1900).jpg  
  inflating: newx64/NotFaces/new (1901).jpg  
  inflating: newx64/NotFaces/new (1902).jpg  
  inflating: newx64/NotFaces/new (1903).jpg  
  inflating: newx64/NotFaces/new (1904).jpg  
  inflating: newx64/NotFaces/new (1905).jpg  
  inflating: newx64/NotFaces/new (1906).jpg  
  inflating: newx64/NotFaces/new (1907).jpg  
  inflating: newx64/NotFaces/new (1908).jpg  
  inflating: newx64/NotFaces/new (1909).jpg  
  inflating: newx64/NotFaces/new (191).jpg  
  inflating: newx64/NotFaces/new (1910).jpg  
  inflating: newx64/NotFaces/new (1911).jpg  
  inflating: newx64/NotFaces/new 

In [ ]:
trainDir = "/content/dataset/train"
testDir = "/content/dataset/test/"
validationDir = "/content/dataset/validation/"
mainDir = "/content/newx64/"
weightsPath = "/content/dataset/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
# Fixed for our Cats & Dogs classes
NUM_CLASSES = 5

# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 9

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [ ]:
# trdata = ImageDataGenerator(preprocessing_function=preprocess_input)
# traindata = trdata.flow_from_directory(directory=trainDir,target_size=(IMAGE_RESIZE, IMAGE_RESIZE))
# vldata = ImageDataGenerator(preprocessing_function=preprocess_input)
# validationdata = trdata.flow_from_directory(directory=validationDir,target_size=(IMAGE_RESIZE, IMAGE_RESIZE))
# tsdata = ImageDataGenerator()
# testdata = tsdata.flow_from_directory(directory=testDir, target_size=(IMAGE_RESIZE, IMAGE_RESIZE))

image_generator = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.3)    

traindata = image_generator.flow_from_directory(batch_size=BATCH_SIZE_TRAINING,
                                                 directory=mainDir,
                                                 shuffle=True,
                                                 target_size=(IMAGE_RESIZE, IMAGE_RESIZE), 
                                                 subset="training",
                                                 class_mode='categorical')

validationdata = image_generator.flow_from_directory(batch_size=BATCH_SIZE_VALIDATION,
                                                 directory=mainDir,
                                                 shuffle=True,
                                                 target_size=(IMAGE_RESIZE, IMAGE_RESIZE), 
                                                 subset="validation",
                                                 class_mode='categorical')

Found 26678 images belonging to 2 classes.
Found 11432 images belonging to 2 classes.


In [ ]:
#VVG16
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [ ]:
#VGG16
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("/content/drive/MyDrive/dataset/vgg16_5.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=16,generator=traindata, validation_data=validationdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
 1/16 [>.............................] - ETA: 10:25 - loss: 0.9518 - accuracy: 0.1800

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
plt.figure(1, figsize = (15,8)) 
    
plt.subplot(221)  
plt.plot(hist.history['accuracy'])  
plt.plot(hist.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
    
plt.subplot(222)  
plt.plot(hist.history['loss'])  
plt.plot(hist.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

plt.show()

In [ ]:
print(traindata.class_indices)

{'Faces': 0, 'NotFaces': 1}
